In [1]:
# !pip uninstall torch torchvision -y
# !pip install torch==1.4.0 torchvision==0.5.0

In [1]:
import pandas as pd
import numpy as np
import os

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [2]:
# from sklearn.datasets import fetch_openml
# mnist = fetch_openml('mnist_784', version=1)
# mnist.keys()

In [3]:
# X, y = mnist['data'], mnist['target']

In [4]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/7, random_state=42)

### Getting Started with Pytorch

In [5]:
from pathlib import Path
import requests

DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

URL = "http://deeplearning.net/data/mnist/"
FILENAME = "mnist.pkl.gz"


if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)


In [34]:
import pickle
import gzip

with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((X_train, y_train), (X_test, y_test), _) = pickle.load(f, encoding="latin-1")

In [35]:
import torch


X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train = (X_train - X_mean) / X_std
X_test = (X_test - X_mean) / X_std


X_train, y_train, X_test, y_test = map(
    torch.tensor, (X_train, y_train, X_test, y_test)
)

X_train = X_train.type(torch.float32)
X_test = X_test.type(torch.float32)
y_train = y_train.type(torch.float32)
y_test = y_test.type(torch.float32)


bs = 64

In [37]:
from torch.utils.data import TensorDataset

train_ds = TensorDataset(X_train, y_train)
test_ds = TensorDataset(X_test, y_test)

In [38]:
from torch.utils.data import DataLoader

def get_data(train_ds, test_ds, bs=64):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(test_ds, batch_size=bs * 2),
    )

In [39]:
train_dl, test_dl = get_data(train_ds, test_ds)

In [47]:
import torch.nn.functional as F
from torch import nn
loss_func = F.cross_entropy
# loss_func = nn.CrossEntropyLoss()

In [48]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [54]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb.type(torch.long))

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

In [55]:
def preprocess(x, y):
    return x.view(-1, 1, 28, 28), y


class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))

train_dl = WrappedDataLoader(train_dl, preprocess)
test_dl = WrappedDataLoader(test_dl, preprocess)

In [118]:
from IPython.core.debugger import set_trace

import numpy as np


def fit(epochs, model, loss_func, opt, train_dl, valid_dl, c=3, p=32):
    epoch_train_loss = []
    epoch_test_loss = []

    for epoch in range(epochs):
        model.train()
        train_losses = []
        train_nums = []
        for xb, yb in train_dl:
            losses, nums = loss_batch(model, loss_func, xb, yb, opt)
            train_losses.append(losses)
            train_nums.append(nums)
        train_loss = np.sum(np.multiply(np.array(train_losses), np.array(train_nums))) / np.sum(train_nums)
        epoch_train_loss.append(train_loss)


        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in test_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        epoch_test_loss.append(val_loss)

        print('epoch #: {}, train_loss: {}, val_loss: {}'.format(epoch, train_loss, val_loss))
        print('train_accuracy: {}, val_accuracy: {}'.format(accuracy(model(X_train.view(-1,c,p,p)), y_train), accuracy(model(X_test.view(-1,c,p,p)), y_test)))
        print('=======================================')

    return epoch_train_loss, epoch_test_loss


In [57]:


class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)

In [58]:
model = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Lambda(lambda x: x.view(x.size(0), -1)),
#     nn.Flatten(),
)

from torch import optim

lr = 0.5  # learning rate
epochs = 15
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

fit(epochs, model, loss_func, opt, train_dl, test_dl, c=1, p=28)

epoch #: 0, train_loss: 1.0377809502601623, val_loss: 0.686959276008606


KeyboardInterrupt: 

### CNNs for CIFAR-10

In [89]:
from sklearn.datasets import fetch_openml
cifar_10_small = fetch_openml('CIFAR_10_Small', version=1)
cifar_10_small.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [109]:
X, y = cifar_10_small['data'], cifar_10_small['target']

In [110]:
X.shape

(20000, 3072)

In [111]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [112]:
# import torchvision
# import torchvision.transforms as transforms

# transform = transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [113]:
import torch


X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train = (X_train - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

y_train = y_train.astype(np.float)
y_test = y_test.astype(np.float)


X_train, y_train, X_test, y_test = map(
    torch.tensor, (X_train, y_train, X_test, y_test)
)

X_train = X_train.type(torch.float32)
X_test = X_test.type(torch.float32)
y_train = y_train.type(torch.float32)
y_test = y_test.type(torch.float32)

bs = 64

In [114]:
X_train.shape

torch.Size([15000, 3072])

In [115]:
train_ds = TensorDataset(X_train, y_train)
test_ds = TensorDataset(X_test, y_test)

train_dl, test_dl = get_data(train_ds, test_ds)

In [116]:
def preprocess_cifar(x, y):
    return x.view(-1, 3, 32, 32), y


train_dl = WrappedDataLoader(train_dl, preprocess_cifar)
test_dl = WrappedDataLoader(test_dl, preprocess_cifar)

In [ ]:
model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(64, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Lambda(lambda x: x.view(x.size(0), -1)),
#     nn.Flatten(),
)

# import torch.nn as nn
# import torch.nn.functional as F


# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
#         self.pool = nn.MaxPool2d(2)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
#         self.conv3 = nn.Conv2d(64, 16, kernel_size=3, stride=2, padding=1)
#         self.avgpool = nn.AdaptiveAvgPool2d(1)
#         self.flatten = nn.Flatten()
    
#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = self.avgpool(F.relu(self.conv3(x)))
#         x = self.flatten(x)
#         return x

# model = Net()


from torch import optim

momentum = 0.9
lr = 0.5  # learning rate
epochs = 10
opt = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

train_losses, test_losses = fit(epochs, model, loss_func, opt, train_dl, test_dl, c=3, p=32)

epoch #: 0, train_loss: 2.6239074687957764, val_loss: 2.579810043334961
train_accuracy: 0.0963333323597908, val_accuracy: 0.10580000281333923
epoch #: 1, train_loss: 2.5983553305308025, val_loss: 2.578149006652832


In [ ]:
plt.plot(train_losses, range(len(train_losses)), 'b-', legend='train' )
plt.plot(test_losses, range(len(test_losses)), 'g--', legend='test')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Model 1: train_vs_test')
plt.legend()
plt.show()

In [ ]:
model2 = nn.Sequential(
    nn.Conv2d(3, 128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(64*16*16, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

from torch import optim

momentum = 0.9
lr = 0.5  # learning rate
epochs = 10
opt = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

train_losses, test_losses = fit(epochs, model, loss_func, opt, train_dl, test_dl, c=3, p=32)

In [ ]:
plt.plot(train_losses, range(len(train_losses)), 'b-', legend='train' )
plt.plot(test_losses, range(len(test_losses)), 'g--', legend='test')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Model 2: train_vs_test')
plt.legend()
plt.show()